# Espaço de desafios

Notebook para o desenvolvimento de códigos de desafio do curso Data Visualization: estilização de tabelas com Python.

## Intro
Vamos praticar o uso da visualização de tabelas através do Pandas. Para a prática, vamos utilizar outro conjunto de dados que está disponível para download: [loja_livro_filmes.csv](https://cdn3.gnarususercontent.com.br/3056-data-visualization/loja_livro_filmes.csv).

Nele, temos um relatório de custos de venda de livros e filmes de uma loja. Durante os desafios, a missão será trabalhar com os dados para fazer tabelas que podem se adequar a necessidade de diversas análises que podem ser feitas da loja. Para um melhor entendimento dos dados, abaixo está o que representa cada dado nas colunas.

### Dicionário de variáveis

* **pais**: país de destino do produto.
* **id_cliente**: código de identificação do cliente solicitante.
* **data_pedido**: data de solicitação do produto por parte do cliente.
* **data_chegada**: data de chegada do produto ao endereço de entrega informado pelo cliente.
* **tipo_compra**: categoria que se enquadra a compra, podendo ser *Normal* ou *Devolução*.
* **numero_pedido**: código de identificação da compra realizada.
* **tamanho_pacote**: tamanho do pacote para envio do pedido.
* **unidades**: quantidade de unidades compradas no pedido.
* **custo_empacotamento**: valor do custo da loja para empacotar o produto.
* **custo_envio**: valor do custo da loja para enviar o produto.
* **tipo_desconto**: categoria do tipo de desconto foi aplicado.
* **categoria**: categoria de tipo de produto.
* **tipo_consumo**: nível de consumo do produto por tempo de uso.
* **tipo_cliente**: categoria de tipo de cliente.
* **custo_produto**: valor total do custo que o pedido resulta para a loja.

## Hora da prática 1

O primeiro desafio é construir uma visualização que permita **identificar quais países mais solicitaram produtos por pedidos** para que assim possa ser feito um estudo quanto a distribuição e logística de produtos.

In [1]:
## Importar o pandas e ler o arquivo
import pandas as pd
df = pd.read_csv('loja_livro_filmes.csv')

## Preparar os dados
# Agrupar a soma das unidades por pais e ordenar os valores
df_paises = df.groupby(['pais'])['unidades'].sum().nlargest(9).copy()
df_paises = df_paises.reset_index()

# Alterar os nomes das colunas e ajustar o ranqueamento
df_paises.columns = ['País', 'Unidades pedidas']
df_paises['Rank'] = df_paises.index + 1
df_paises.set_index('Rank', inplace=True)
df_paises

## Criar a visualização
s_pais = df_paises.style
# Formatar o texto para adicionar a unidade de medida
s_pais = s_pais.format({'Unidades pedidas': '{} prod'})
s_pais

,País,Unidades pedidas
Rank,,
1,Argentina,5591 prod
2,Equador,5566 prod
3,Paraguai,5564 prod
4,Peru,5550 prod
5,Venezuela,5529 prod
6,Colômbia,5504 prod
7,Brasil,5473 prod
8,Uruguai,5377 prod
9,Chile,5304 prod


## Hora da prática 2

O objetivo desse desafio é construir uma visualização que permita **identificar quais categorias de produto geraram mais e menos custos para a loja**.

In [2]:
## Importar o pandas e ler o arquivo
import pandas as pd
df = pd.read_csv('loja_livro_filmes.csv')

## Preparar os dados
# Agrupar a soma dos custos dos pedidos por categoria
df_produto_custo = df.groupby(['categoria'])[['custo_produto']].sum()
# Ajustar o nome da coluna e index
df_produto_custo.index.name = 'Categoria produto'
df_produto_custo.columns = ['Custo de pedidos']
df_produto_custo

## Criar a visualização
s_produto = df_produto_custo.style
# Destacar elementos
s_produto.format('R$ {:,.2f}')\
         .highlight_max(color='#F16165')\
         .highlight_min(color='lightgreen')

,Custo de pedidos
Categoria produto,
BlueRay,"R$ 14,191.36"
CD,"R$ 10,514.28"
Coleção,"R$ 73,128.59"
Livro,"R$ 78,213.80"


## Hora da prática 3

O desafio é construir uma visualização que permita **informar a quantidade e distribuição de pedidos por tipo de desconto**. Isso irá permitir que os setores da empresa entendam a demanda de produtos em cada ação promocional.

In [ ]:
## Importar o pandas e ler o arquivo
import pandas as pd
df = pd.read_csv('loja_livro_filmes.csv')

## Preparar os dados
# Ter o DataFrame fazendo a contagem dos pedidos em cada categoria de desconto
df_descontos = pd.DataFrame(df['tipo_desconto'].value_counts())
# Ajustar o nome da coluna e index
df_descontos.columns = ['N° pedidos']
df_descontos.index.name = 'Tipo desconto'
# Coletamos a distribuição da quantidade de pedidos por porcentagem
porcentagem = df_descontos['N° pedidos'].to_numpy()
porcentagem = 100 * porcentagem/porcentagem.sum()
df_descontos['Distribuição de pedidos'] = porcentagem
df_descontos

## Criar a visualização
s_descontos = df_descontos.style
# Utilizar propriedades CSS para formatar a visualização
cabecalho = {
    'selector': 'th',
    'props': 'font-weight: bold; font-family: Arial; text-align: right; background-color: white; color: black;'
}
celulas = {
    'selector': 'td',
    'props': 'background-color: white; color: black;'
}
s_descontos.set_table_styles([cabecalho, celulas], overwrite=False)
# Aplicar a visualização de gráfico
# os parâmetros height e width permitem alterar a proporção do tamanho do gráfico em relação a padrão 100
s_descontos.format({'Distribuição de pedidos': '{:.2f} %'})\
           .bar(subset='Distribuição de pedidos', vmin = 0, vmax = 100.0, color = '#093364',
                height=50,width=60)

## Hora da prática 4

Nosso desafio agora é fornecer uma visualização que permita **mostrar o tempo médio de entrega de pedidos durante os meses dos anos de 2013 a 2015 por cada país**. Isso vai permitir analisar o desempenho da parte de transportes da empresa durante os anos e traçar um planejamento de melhorias.

In [1]:
## Importar o pandas e ler o arquivo
import pandas as pd
df = pd.read_csv('loja_livro_filmes.csv')

## Preparar os dados
# Tornar 'data_chegada' e 'data_pedido' colunas datetime
df['data_chegada'] = pd.to_datetime(df['data_chegada'])
df['data_pedido'] = pd.to_datetime(df['data_pedido'])

## Construir cópia de um DataFrame para adicionar novas colunas de dados
df_datas = df.copy()
df_datas = df_datas.sort_values('data_pedido')
df_datas = df_datas.reset_index(drop=True)

# Coletar apenas informações a partir do ano de 2013
df_datas = df_datas[df_datas['data_pedido'] >= '2013-01-01']

# Adicionar coluna com os meses formatados
df_datas['meses'] = df_datas['data_pedido'].dt.strftime('%Y - %b')

# Adicionar uma coluna com o tempo de entrega entre pedido e chegada
tempo_demora = (df_datas['data_chegada'] - df_datas['data_pedido']).dt.days
df_datas['tempo_entrega'] = tempo_demora

# Criar a estrutura em DataFrame
entregas = df_datas.pivot_table(index = 'pais', columns = 'meses', values = 'tempo_entrega' , aggfunc = 'mean', sort=False)

## Criar a visualização
s_entregas = entregas.style.format('{:,.2f}')
s_entregas.set_sticky(axis="index")

meses,2013 - Apr,2013 - Aug,2013 - Dec,2013 - Feb,2013 - Jan,2013 - Jul,2013 - Jun,2013 - Mar,2013 - May,2013 - Nov,2013 - Oct,2013 - Sep,2014 - Apr,2014 - Aug,2014 - Dec,2014 - Feb,2014 - Jan,2014 - Jul,2014 - Jun,2014 - Mar,2014 - May,2014 - Nov,2014 - Oct,2014 - Sep,2015 - Apr,2015 - Aug,2015 - Dec,2015 - Feb,2015 - Jan,2015 - Jul,2015 - Jun,2015 - Mar,2015 - May,2015 - Nov,2015 - Oct,2015 - Sep
pais,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Chile,351.70,207.92,285.69,341.24,90.21,307.23,99.73,470.11,195.55,80.67,137.70,321.06,252.60,358.42,329.43,483.75,633.85,149.89,469.79,658.22,309.82,327.12,341.12,403.83,424.06,447.47,254.91,549.07,530.29,432.64,353.83,606.28,423.43,406.92,502.29,314.11
Uruguai,251.06,161.82,249.60,201.27,180.88,217.79,555.25,146.52,12.50,222.76,90.00,117.33,343.81,487.89,624.85,420.64,494.30,379.56,555.58,513.40,558.94,545.12,542.15,751.57,395.70,280.59,184.42,388.35,313.26,349.48,360.04,231.84,502.35,434.12,453.33,346.00
Venezuela,221.54,393.40,264.86,267.50,216.05,282.42,315.77,219.40,180.88,370.05,187.14,405.16,713.86,541.44,336.89,529.82,550.70,336.91,441.38,447.92,735.25,279.88,441.33,183.55,619.32,236.68,187.75,739.95,510.44,443.09,442.91,307.56,413.48,271.11,327.42,258.38
Argentina,481.17,171.75,437.70,314.21,321.91,351.89,309.32,356.00,346.42,513.63,149.39,342.20,304.42,465.56,373.05,527.11,419.76,570.82,592.68,482.53,536.00,398.42,385.65,452.94,467.70,506.90,293.12,465.42,447.40,374.68,529.89,324.67,264.59,317.96,299.18,437.33
Equador,206.21,252.24,249.20,16.93,718.81,317.88,233.00,80.67,178.38,251.87,479.15,205.38,402.00,424.53,370.35,500.07,588.10,275.65,263.50,526.46,227.65,479.41,261.06,306.11,277.61,425.28,151.00,485.79,365.41,279.64,566.53,338.86,440.35,335.05,238.42,337.30
Colômbia,136.73,643.80,432.50,293.07,339.46,377.93,211.05,328.85,580.14,16.00,570.00,139.30,562.21,582.05,650.36,372.76,462.00,529.55,621.11,608.53,471.77,441.25,341.79,513.67,328.94,433.91,393.78,204.57,305.96,399.00,438.86,452.52,433.17,149.95,462.63,435.48
Brasil,382.64,266.13,241.94,520.55,238.74,181.22,232.25,406.64,454.87,279.78,213.67,272.46,365.77,454.21,144.00,652.61,532.88,430.80,291.75,328.92,479.71,416.86,434.72,534.81,432.07,455.69,241.50,380.69,284.03,310.94,222.56,431.16,375.54,391.48,394.18,392.79
Paraguai,272.29,285.29,375.62,384.73,216.86,120.42,263.57,495.88,345.21,81.72,351.67,506.47,473.91,304.64,237.08,436.29,718.06,332.13,369.50,461.08,665.82,387.39,596.90,506.08,372.72,405.35,107.25,613.50,314.32,607.22,354.54,510.65,463.72,203.30,354.65,261.52
Peru,193.47,371.07,218.32,379.05,283.48,333.33,216.08,436.15,167.53,290.62,319.38,280.93,432.79,390.50,175.80,440.88,311.59,493.86,398.39,493.32,631.00,464.33,336.21,598.83,393.83,556.28,323.09,388.39,725.22,330.58,332.81,470.48,499.64,325.76,204.71,471.40


## Hora da prática 5

A empresa que estamos trabalhando distribui produtos por toda América do Sul para várias categorias de clientes. Nosso desafio envolve uma visualização que permita **separar os gastos de envio de cada produto (custo total) por tipo de cliente**, de modo que seja possível entender o tipo de produto que mais gerou custos de envio em cada tipo de cliente. Além disso, é interessante também mostrar a **distribuição desses custos por país enviado**, permitindo entender também os gastos por país.

In [2]:
## Importar o pandas e ler o arquivo
import pandas as pd
df = pd.read_csv('loja_livro_filmes.csv')

## Preparar os dados

# Criar um DataFrame alternativo para ter um nome de coluna coerente com a visualização
df_alteracao = df.rename(columns={'pais': 'Custo por país ($)'})

# Agrupar os dados de tipo de cliente, com a categoria de produtos e os países que esses clientes importam o produto
df_clientes = df_alteracao.pivot_table(index=['tipo_cliente', 'categoria'], columns = 'Custo por país ($)', values = 'custo_produto', aggfunc = 'sum')

# Renomear o cabeçalho para nomes mais coerentes
df_clientes.rename_axis(index={'tipo_cliente': 'Tipo cliente', 'categoria': 'Categoria produto'}, inplace = True)

# Adicionar uma coluna que mostra a soma dos custos por tipo de cliente e categoria de produto
df_clientes['Total ($)'] = df_clientes.sum(axis=1)

## Criar a visualização
s_clientes = df_clientes.style.format('{:,.2f}')

# Criar os estilos base para a tabela, a partir da padronização das fontes, plano de fundo para branco e o alinhamento do cabeçalho para o topo
tabela = {
    'selector': 'td, th',
    'props': 'font-weight: normal; font-family: Arial; text-align: right; background-color: white;'
}
# Alinhar verticalmente o cabeçalho para o topo permite a pessoa observadora identificar onde começa a hierarquização dos grupos B2B e B2C
cabecalho = {
    'selector': 'th',
    'props': 'vertical-align: top'
}
s_clientes.set_table_styles([tabela, cabecalho])

# Adicionar linha de separação do cabeçalho superior
# Destacar as categorias que mais geraram custo com a aplicação de negrito nas fontes
s_clientes.set_table_styles({
    ('B2B', 'BlueRay'): [{'selector': 'th', 'props': 'border-top: 1px solid #181818'},
              {'selector': 'td', 'props': 'border-top: 1px solid #181818'}],
    ('B2B', 'Livro'): [{'selector': 'th', 'props': 'font-weight: bold'}],
    ('B2C', 'Livro'): [{'selector': 'th', 'props': 'font-weight: bold'}],
}, overwrite=False, axis=1)

# Destacar os elementos em cada tipo de cliente que mais geraram custos com uma seleção
s_clientes.set_table_styles({
    'Total ($)': [{
        'selector': '.true',
        'props': 'font-weight: bold'
    }]
},overwrite=False,axis=0)

cores_coluna = pd.DataFrame(['false','false','false','true','false','false','false','true'],index= df_clientes['Total ($)'].index,
                            columns = ['Total ($)'])
# Visualização final
s_clientes.set_td_classes(cores_coluna)